# Setup

In [1]:
import os
from Bio import SeqIO, Align
from Bio.Seq import Seq
import pandas as pd
import time

# Utility functions

In [2]:
def get_signature_seq_type_n_orientation(N_signature_seq, C_signature_seq, filtered_read):

    all_signature_seqs = {
            "N_sign_FW": N_signature_seq,
            "C_sign_FW": C_signature_seq,
            "N_sign_RV": N_signature_seq.reverse_complement(),
            "C_sign_RV": C_signature_seq.reverse_complement()
    }

    outcome_sig_seq_type_n_orientations = list()

    for signature_seq_type_n_dir, signature_seq in all_signature_seqs.items():
        alignments = aligner.align(filtered_read, signature_seq)
        for alignment in alignments:
            if alignment.score == len(signature_seq):
                outcome_sig_seq_type_n_orientations.append(signature_seq_type_n_dir)
            
    return outcome_sig_seq_type_n_orientations


def get_filtered_read_in_inserted_CDS_orientation(filtered_read, sig_seq_type_n_orientation):
    
    if sig_seq_type_n_orientation == "N_sign_FW" or sig_seq_type_n_orientation == "C_sign_RV":
        return filtered_read
    elif sig_seq_type_n_orientation == "N_sign_RV" or sig_seq_type_n_orientation == "C_sign_FW":
        return filtered_read.reverse_complement()
    else:
        print("Input Error")
        raise


def get_signature_seq_in_inserted_CDS_orientation(N_signature_seq, C_signature_seq, sig_seq_type_n_orientation):
    
    signature_seq_opts = {
        'N': N_signature_seq,
        'C': C_signature_seq.reverse_complement()
    }
    N_or_C_sig_seq = sig_seq_type_n_orientation.split("_")[0]
    return signature_seq_opts[N_or_C_sig_seq]


def get_best_alignment_of_extracted_seq_to_CDS(CDS_seq, seq_for_alignment_to_CDS):
    '''Find perfect_alignment and return if it is unique, otherwise output False'''
    
    if len(seq_for_alignment_to_CDS) == 0:
        return False
    
    query_seqs = {
        'FW': seq_for_alignment_to_CDS,
        'RV': seq_for_alignment_to_CDS.reverse_complement()
    }
    perfect_alignments = list()
    for query_seq_orientation, query_seq in query_seqs.items():
        alignments = aligner.align(CDS_seq, query_seq)
        for alignment in alignments:
            if alignment.score == len(seq_for_alignment_to_CDS):
                perfect_alignments.append((query_seq_orientation, alignment))
                
    # if there is more than one perfect alignment, then the extract seq is too short
    if len(perfect_alignments) == 1:
        return perfect_alignments[0]
    else:
        return False

Main function to infer split sites.
Depends on utility functions.

In [3]:
def infer_split_site(aligner, input_seq_iterator, N_signature_seq, C_signature_seq, CDS_seq, extracted_seq_len):

    aligned_reads_df = pd.DataFrame()

    # --------------Debugging scripts below--------------
    records_subset = list()
    for i in range(20):
        records_subset.append(next(input_seq_iterator))
    
    for i, rec in enumerate(records_subset): # for the purpose of illustration, only the 1st 20 examples are ran
    # Comment out the scripts above and uncomment the line below for complete analysis
    # --------------Debugging scripts above--------------

#     for i, rec in enumerate(input_seq_iterator): # Uncomment this line for complete analysis

        filtered_read = rec.seq

        # Step 1: Flip all flitered read sequences into the orientation of the ***inserted CDS*** for visualization and debugging

        sig_seq_type_n_orientations = get_signature_seq_type_n_orientation(N_signature_seq, C_signature_seq, filtered_read)

        # If more than one signature sequences can perfectly align on the same read, double transposition might have happened
        if len(sig_seq_type_n_orientations) != 1:
            continue

        filtered_read_in_inserted_CDS_orientation = get_filtered_read_in_inserted_CDS_orientation(filtered_read, 
                                                                                              sig_seq_type_n_orientations[0])
        signature_seq_in_inserted_CDS_orientation = get_signature_seq_in_inserted_CDS_orientation(N_signature_seq, C_signature_seq,
                                                                            sig_seq_type_n_orientations[0])

        alignment_sign_to_filtered = aligner.align(filtered_read_in_inserted_CDS_orientation, signature_seq_in_inserted_CDS_orientation)

        # --------------Debugging scripts below--------------
        print("==========================")
        print("Record # =", i)
        print("Record ID =", rec.id)
        print("signature seq. type and orientation =", sig_seq_type_n_orientations[0], "\n")
        print(alignment_sign_to_filtered[0])
        print("alignment score =", alignment_sign_to_filtered[0].score, "\n\n")
        # --------------Debugging scripts above--------------

        signature_seq_type = sig_seq_type_n_orientations[0].split("_")[0]

        # Step 2: Extract short DNA sequence adjacent to the extein junctions, N and C separated method

        # Work out where signature seq is within the filtered read
        sig_start,sig_end = alignment_sign_to_filtered[0].path[0][0], alignment_sign_to_filtered[0].path[1][0]



        # Extract a short sequence adjacent to the extein residues
        if signature_seq_type == "N":
            seq_for_alignment_to_CDS = filtered_read_in_inserted_CDS_orientation[(sig_start-extracted_seq_len):sig_start]
        elif signature_seq_type == "C":
            seq_for_alignment_to_CDS = filtered_read_in_inserted_CDS_orientation[(sig_end):(sig_end+extracted_seq_len)]
        else:
            print("Error")
            raise

        # --------------Debugging scripts below--------------
        print("Signature seq. start and end positions on reads = )", sig_start, ", " ,sig_end, ")")
        print("Extracted seq. for alignment to CDS =", seq_for_alignment_to_CDS)
        # --------------Debugging scripts above--------------


        # Step 3: Align the extracted sequence to the CDS

        # insertion can happens in both orientation of the CDS
        # also, if the extracted sequence is too short there will be multiple hits
        # these cases are considered in the function below:

        best_alignment = get_best_alignment_of_extracted_seq_to_CDS(CDS_seq, seq_for_alignment_to_CDS)

        if best_alignment == False:        
            # --------------Debugging scripts below--------------
            print("No unique match to CDS")
            # --------------Debugging scripts above--------------
            continue

        insertion_orientation, alignment_extracted_seq_to_CDS = best_alignment

        # --------------Debugging scripts below--------------
        print(alignment_extracted_seq_to_CDS)
        print("Insertion orientation =", insertion_orientation)
        # --------------Debugging scripts above--------------

        # Step 4: DNA and protein split site identification

        # This part might not be straightforward from the code, but essentially:
        # Deduplication of 5 bp is always defined on the 5' end regardless of insertion orientation, AND,
        # Both (FW inserted CDS N-terminal end) and (RV inserted C-terminal end) will give info on 5' end split site & vice-versa

        # 5' end case
        if (insertion_orientation == "RV" and signature_seq_type == "C") or (insertion_orientation == "FW" and signature_seq_type == "N"):
            dna_split_site_middle = alignment_extracted_seq_to_CDS.path[1][0] - 4.5 - 60 # 60 bp to be corrected from the extension of CDS at 5'
        # 3' end case
        elif (insertion_orientation == "RV" and signature_seq_type == "N") or (insertion_orientation == "FW" and signature_seq_type == "C"):
            dna_split_site_middle = alignment_extracted_seq_to_CDS.path[0][0] + 0.5 - 60
        else:
            print("ERROR")
            raise

        productive_insertion = False

        if insertion_orientation == "RV":
            aa_split_site_middle = 0
        elif insertion_orientation == "FW":
            aa_split_site_middle = round((dna_split_site_middle + 2.5) / 3 - 0.5, 2)
            if (dna_split_site_middle + 2.5) % 3 == 0:
                productive_insertion = True
        else:
            print("ERROR")
            raise

        # --------------Debugging scripts below--------------
        print("DNA split site =", dna_split_site_middle)
        print("aa split site =", aa_split_site_middle)
        print("productive insertion =", productive_insertion)
        # --------------Debugging scripts above--------------

        aligned_read_pd_entry = pd.DataFrame({
            "fastq_id": [rec.id],
            "ssto": [sig_seq_type_n_orientations[0]],
            "insertion_orientation": [insertion_orientation],
            "dna_split_site_middle": [dna_split_site_middle],
            "aa_split_site_middle": [aa_split_site_middle],
            "productive_insertion": [productive_insertion]
        })

        aligned_reads_df = aligned_reads_df.append(aligned_read_pd_entry)
        
    return aligned_reads_df

# Parameters, variables, target CDS for alignment

In [4]:
# Set up parameters for aligner to enforce perfect match
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.open_gap_score = -10
aligner.extend_gap_score = -10

In [5]:
# Define signature sequences
# Both the length of signature sequence and length of extracted sequences for perfect match required was defined as 12 bp
extracted_seq_len = 12

M86_N_seq_30bp = Seq("TGGATGCATCTCGGGAGATAGTTTGATCAG")
M86_C_seq_30bp = Seq("TGAGTTATGTACAATGATGTCATTGGCGAC") # in reverse complement of the M86 CDS
M86_N_sig = M86_N_seq_30bp[:extracted_seq_len]
M86_C_sig = M86_C_seq_30bp[:extracted_seq_len]

In [6]:
# Get CDS from a fasta file  
# Note that CDS here is actually from 60 bp before start codon to 60 bp after stop codon

target_CDS_fasta_filename = "target_protein_CDS_plus.fa"
read_handle = open(target_CDS_fasta_filename, "r")

target_CDS_seq_records = dict()
for record in SeqIO.parse(read_handle, "fasta"):
    target_CDS_seq_records.update({record.id.split("_")[0]: record.seq})

# Files

In [7]:
filenames = [
     "example_IBM_NGS_filtered_1_mCherry_1.fq" # Expand this list for multiple files handling, or use fnmatch
]

# Execution

Note that this step is very time consuming and can take up to 16 hours for just one file.

By the end of execution, the time required to process one file is recorded in `./NGS_alignment_time.txt`

In [8]:
f = open("NGS_alignment_time.txt", "a+") # for recording time needed on PC

for input_filename in filenames:

    library_n_ID = input_filename.split("filtered_")[1].split(".")[0]
    target_protein_name = library_n_ID.split("_")[1].split("_")[0]
    
    CDS_seq = target_CDS_seq_records[target_protein_name]
    
    output_filename = "example_IBM_NGS_aligned_reads_" + library_n_ID + ".csv" # change this line to customize output filename
    output_path = os.path.join("results_per_fastq", output_filename)
    
    # Set signature sequences
    N_signature_seq = M86_N_sig
    C_signature_seq = M86_C_sig
        
    start = time.time()
    
    input_file_path = os.path.join("filtered_fastq_files", input_filename)
    read_handle = open(input_file_path, "r")
    input_seq_iterator = SeqIO.parse(read_handle, "fastq")
    aligned_reads_df = infer_split_site(aligner, input_seq_iterator, N_signature_seq, C_signature_seq, CDS_seq, extracted_seq_len)
    aligned_reads_df.to_csv(output_path)
    read_handle.close()
    
    end = time.time()
    
    time_record = "Fastq file = " + library_n_ID + ", Time elapsed =" + str(end - start) + "\n"
    f.write(time_record)
    
    # --------------Debugging scripts below--------------
    print("==========================Analysis Summary==========================")
    print("target protein =", target_protein_name)
    print("CDS seq =", CDS_seq)
    print("input path =", input_file_path)
    print("input filename =", input_filename)
    print("output filename =", output_filename)
    print("output path =", output_path)
    print("N signature Seq =", N_signature_seq)
    print("C signature Seq =", C_signature_seq)
    print(time_record)
    # --------------Debugging scripts above--------------
    
f.close()

Record # = 0
Record ID = A00627:119:HJ57YDSXY:1:1101:12084:1157
signature seq. type and orientation = N_sign_FW 

TACAAGGTGAAGCTGCGCGGCACCAACTTCCCCTCCGATGGATGCATCTCGGGAGATAGTTTGATCAGCTTGGCGAGCACAGGGAAAAGAGTTCCTATTAAGGATTTGTTAGGCGAAAAAGATTTTGAAATATGGGCAATTAATGAACAG
                                      ||||||||||||                                                                                                    
                                      TGGATGCATCTC                                                                                                    

alignment score = 12.0 


Signature seq. start and end positions on reads = ) 38 ,  50 )
Extracted seq. for alignment to CDS = CTTCCCCTCCGA
TTTTTATCGCAACTCTCTACTGTTTCTCCATTACTAGAGATTAAAGAGGAGAAATACTAGATGGTGAGCAAGGGCGAAGAAGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGCACATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAGGGCGAGGGCGAGGGCCGCCCCTACGAGGGCACCCAGACCGCCAAGCTGAAGGTGACCAAGGGTGGCCCCCTGCCCTTCGCCTGGGACATCCTGTCCCCTCAGTTCATGTACGGCTCCAAGGCCTACG

# Inspect output data

In [9]:
pd.read_csv("results_per_fastq/example_IBM_NGS_aligned_reads_1_mCherry_1.csv", index_col=0)

,fastq_id,ssto,insertion_orientation,dna_split_site_middle,aa_split_site_middle,productive_insertion
0,A00627:119:HJ57YDSXY:1:1101:12084:1157,N_sign_FW,FW,405.5,135.50,True
0,A00627:119:HJ57YDSXY:1:1101:7735:1172,N_sign_RV,RV,309.5,0.00,False
0,A00627:119:HJ57YDSXY:1:1101:13602:1219,C_sign_RV,FW,375.5,125.50,True
0,A00627:119:HJ57YDSXY:1:1101:7374:1454,N_sign_FW,FW,545.5,182.17,False
0,A00627:119:HJ57YDSXY:1:1101:10203:1470,C_sign_RV,RV,119.5,0.00,False
0,A00627:119:HJ57YDSXY:1:1101:10438:1595,N_sign_FW,RV,552.5,0.00,False
0,A00627:119:HJ57YDSXY:1:1101:14751:1861,N_sign_FW,RV,628.5,0.00,False
0,A00627:119:HJ57YDSXY:1:1101:25418:1924,N_sign_FW,RV,650.5,0.00,False
0,A00627:119:HJ57YDSXY:1:1101:8838:1955,N_sign_RV,FW,85.5,28.83,False
0,A00627:119:HJ57YDSXY:1:1101:10610:1955,N_sign_FW,FW,631.5,210.83,False
